In [1]:
import pandas as pd
import numpy as np
import openpyxl

In [2]:
import polars as pl
from datetime import date
q = (
    pl.scan_csv('./csv_data/INS-W_1/FeatureData/bluetooth.csv')
    .select(pl.col("*"))
    .cast({"date": pl.Date})
    .drop("")
    .with_columns(pl.col("pid").str.replace_all("INS-W_",""))
    .cast({"pid": pl.Int32})
    .select(pl.exclude(pl.String))
)

data = q.collect()
print(data.glimpse(return_as_string=True))

Rows: 14260
Columns: 596
$ pid                                                 <i32> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1
$ date                                               <date> 2018-04-03, 2018-04-04, 2018-04-05, 2018-04-06, 2018-04-07, 2018-04-08, 2018-04-09, 2018-04-10, 2018-04-11, 2018-04-12
$ f_blue:phone_bluetooth_rapids_countscans:14dhist    <f64> None, None, None, None, None, None, None, None, None, None
$ f_blue:phone_bluetooth_rapids_uniquedevices:14dhi…  <f64> None, None, None, None, None, None, None, None, None, None
$ f_blue:phone_bluetooth_rapids_countscansmostuniqu…  <f64> None, None, None, None, None, None, None, None, None, None
$ f_blue:phone_bluetooth_doryab_countscansall:14dhi…  <f64> None, None, None, None, None, None, None, None, None, None
$ f_blue:phone_bluetooth_doryab_uniquedevicesall:14…  <f64> None, None, None, None, None, None, None, None, None, None
$ f_blue:phone_bluetooth_doryab_meanscansall:14dhist  <f64> None, None, None, None, None, None, None, None, None

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data_normalized = pl.from_numpy(
    scaler.fit_transform(data.select(pl.exclude([pl.Date, pl.Int32]))), schema=data.select(pl.exclude([pl.Date, pl.Int32])).columns
    )

shape: (50, 594)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ f_blue:ph ┆ f_blue:ph ┆ f_blue:ph ┆ f_blue:ph ┆ … ┆ f_blue:ph ┆ f_blue:ph ┆ f_blue:ph ┆ f_blue:p │
│ one_bluet ┆ one_bluet ┆ one_bluet ┆ one_bluet ┆   ┆ one_bluet ┆ one_bluet ┆ one_bluet ┆ hone_blu │
│ ooth_rapi ┆ ooth_rapi ┆ ooth_rapi ┆ ooth_dory ┆   ┆ ooth_dory ┆ ooth_dory ┆ ooth_dory ┆ etooth_d │
│ ds_…      ┆ ds_…      ┆ ds_…      ┆ ab_…      ┆   ┆ ab_…      ┆ ab_…      ┆ ab_…      ┆ oryab_…  │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│ f64       ┆ f64       ┆ f64       ┆ f64       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 0.014769  ┆ 0.005257  ┆ 0.029324  ┆ 0.014769  ┆ … ┆ NaN       ┆ NaN       ┆ NaN       ┆ NaN      │
│ 0.000671  ┆ 0.000515  ┆ 0.0       ┆ 0.000671  ┆ … ┆ NaN       ┆ NaN     

In [ ]:
# split data test/train
from sklearn.model_selection import train_test_split
train, test = train_test_split(data_normalized, test_size=0.33, random_state=1)
print(train.sample(50))

In [ ]:
rng = np.random.RandomState(0)

from sklearn.ensemble import RandomForestRegressor

from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer, KNNImputer, SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline

N_SPLITS = 4
regressor = RandomForestRegressor(random_state=0)

# Define imputer
imputer = IterativeImputer(random_state=36, max_iter=10, add_indicator=True)

# fit on the dataset
imputer.fit(data_normalized)

df_imputed = imputer.transform(data_normalized)

c:\Users\darkenral\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\impute\_iterative.py:895: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [15]:
normalised_data = pl.from_numpy(df_imputed)
normalised_data.head(10)

column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19,column_20,column_21,column_22,column_23,column_24,column_25,column_26,column_27,column_28,column_29,column_30,column_31,column_32,column_33,column_34,column_35,column_36,…,column_557,column_558,column_559,column_560,column_561,column_562,column_563,column_564,column_565,column_566,column_567,column_568,column_569,column_570,column_571,column_572,column_573,column_574,column_575,column_576,column_577,column_578,column_579,column_580,column_581,column_582,column_583,column_584,column_585,column_586,column_587,column_588,column_589,column_590,column_591,column_592,column_593
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0.041616,0.03349,0.088145,0.041616,0.03349,0.06327,0.084196,0.099077,0.08815,0.08815,0.002402,0.162759,0.162759,0.054045,0.15312,0.076186,0.062632,0.097153,0.088123,0.088028,0.062696,0.061384,0.0,0.037365,0.033325,0.070038,0.051346,0.06309,0.039576,0.000986,0.033666,0.164798,0.162759,0.472753,0.541354,0.464842,0.472753,…,0.000211,0.012419,0.099663,0.027222,0.470575,0.403556,0.425825,0.470577,0.403548,0.390389,0.443344,0.458253,0.425836,0.420324,0.016301,0.093686,0.02285,0.418902,0.463053,0.418689,0.403299,0.416489,0.425708,0.42024,0.424514,0.433089,0.0,0.399025,0.396212,0.428219,0.390085,0.358856,0.341851,0.000225,0.500792,0.085086,0.022837
0.041616,0.03349,0.088145,0.041616,0.03349,0.06327,0.084196,0.099077,0.08815,0.08815,0.002402,0.162759,0.162759,0.054045,0.15312,0.076186,0.062632,0.097153,0.088123,0.088028,0.062696,0.061384,0.0,0.037365,0.033325,0.070038,0.051346,0.06309,0.039576,0.000986,0.033666,0.164798,0.162759,0.472753,0.541354,0.464842,0.472753,…,0.000211,0.012419,0.099663,0.027222,0.470575,0.403556,0.425825,0.470577,0.403548,0.390389,0.443344,0.458253,0.425836,0.420324,0.016301,0.093686,0.02285,0.418902,0.463053,0.418689,0.403299,0.416489,0.425708,0.42024,0.424514,0.433089,0.0,0.399025,0.396212,0.428219,0.390085,0.358856,0.341851,0.000225,0.500792,0.085086,0.022837
0.041616,0.03349,0.088145,0.041616,0.03349,0.06327,0.084196,0.099077,0.08815,0.08815,0.002402,0.162759,0.162759,0.054045,0.15312,0.076186,0.062632,0.097153,0.088123,0.088028,0.062696,0.061384,0.0,0.037365,0.033325,0.070038,0.051346,0.06309,0.039576,0.000986,0.033666,0.164798,0.162759,0.472753,0.541354,0.464842,0.472753,…,0.000211,0.012419,0.099663,0.027222,0.470575,0.403556,0.425825,0.470577,0.403548,0.390389,0.443344,0.458253,0.425836,0.420324,0.016301,0.093686,0.02285,0.418902,0.463053,0.418689,0.403299,0.416489,0.425708,0.42024,0.424514,0.433089,0.0,0.399025,0.396212,0.428219,0.390085,0.358856,0.341851,0.000225,0.500792,0.085086,0.022837
0.041616,0.03349,0.088145,0.041616,0.03349,0.06327,0.084196,0.099077,0.08815,0.08815,0.002402,0.162759,0.162759,0.054045,0.15312,0.076186,0.062632,0.097153,0.088123,0.088028,0.062696,0.061384,0.0,0.037365,0.033325,0.070038,0.051346,0.06309,0.039576,0.000986,0.033666,0.164798,0.162759,0.472753,0.541354,0.464842,0.472753,…,0.000211,0.012419,0.099663,0.027222,0.470575,0.403556,0.425825,0.470577,0.403548,0.390389,0.443344,0.458253,0.425836,0.420324,0.016301,0.093686,0.02285,0.418902,0.463053,0.418689,0.403299,0.416489,0.425708,0.42024,0.424514,0.433089,0.0,0.399025,0.396212,0.428219,0.390085,0.358856,0.341851,0.000225,0.500792,0.085086,0.022837
0.041616,0.03349,0.088145,0.041616,0.03349,0.06327,0.084196,0.099077,0.08815,0.08815,0.002402,0.162759,0.162759,0.054045,0.15312,0.076186,0.062632,0.097153,0.088123,0.088028,0.062696,0.061384,0.0,0.037365,0.033325,0.070038,0.051346,0.06309,0.039576,0.000986,0.033666,0.164798,0.162759,0.472753,0.541354,0.464842,0.472753,…,0.000211,0.012

In [65]:
imputed_data = pd.DataFrame(df_imputed, columns = data_normalized.columns)
imputed_data = pl.from_pandas(imputed_data)
data_temp = data.select(["pid","date"])
imputed_merged = data_temp.hstack(imputed_data)
imputed_merged.sample(20)

pid,date,f_blue:phone_bluetooth_rapids_countscans:14dhist,f_blue:phone_bluetooth_rapids_uniquedevices:14dhist,f_blue:phone_bluetooth_rapids_countscansmostuniquedevice:14dhist,f_blue:phone_bluetooth_doryab_countscansall:14dhist,f_blue:phone_bluetooth_doryab_uniquedevicesall:14dhist,f_blue:phone_bluetooth_doryab_meanscansall:14dhist,f_blue:phone_bluetooth_doryab_stdscansall:14dhist,f_blue:phone_bluetooth_doryab_countscansmostfrequentdevicewithinsegmentsall:14dhist,f_blue:phone_bluetooth_doryab_countscansmostfrequentdeviceacrosssegmentsall:14dhist,f_blue:phone_bluetooth_doryab_countscansmostfrequentdeviceacrossdatasetall:14dhist,f_blue:phone_bluetooth_doryab_countscansleastfrequentdevicewithinsegmentsall:14dhist,f_blue:phone_bluetooth_doryab_countscansleastfrequentdeviceacrosssegmentsall:14dhist,f_blue:phone_bluetooth_doryab_countscansleastfrequentdeviceacrossdatasetall:14dhist,f_blue:phone_bluetooth_doryab_countscansown:14dhist,f_blue:phone_bluetooth_doryab_uniquedevicesown:14dhist,f_blue:phone_bluetooth_doryab_meanscansown:14dhist,f_blue:phone_bluetooth_doryab_stdscansown:14dhist,f_blue:phone_bluetooth_doryab_countscansmostfrequentdevicewithinsegmentsown:14dhist,f_blue:phone_bluetooth_doryab_countscansmostfrequentdeviceacrosssegmentsown:14dhist,f_blue:phone_bluetooth_doryab_countscansmostfrequentdeviceacrossdatasetown:14dhist,f_blue:phone_bluetooth_doryab_countscansleastfrequentdevicewithinsegmentsown:14dhist,f_blue:phone_bluetooth_doryab_countscansleastfrequentdeviceacrosssegmentsown:14dhist,f_blue:phone_bluetooth_doryab_countscansleastfrequentdeviceacrossdatasetown:14dhist,f_blue:phone_bluetooth_doryab_countscansothers:14dhist,f_blue:phone_bluetooth_doryab_uniquedevicesothers:14dhist,f_blue:phone_bluetooth_doryab_meanscansothers:14dhist,f_blue:phone_bluetooth_doryab_stdscansothers:14dhist,f_blue:phone_bluetooth_doryab_countscansmostfrequentdevicewithinsegmentsothers:14dhist,f_blue:phone_bluetooth_doryab_countscansmostfrequentdeviceacrosssegmentsothers:14dhist,f_blue:phone_bluetooth_doryab_countscansmostfrequentdeviceacrossdatasetothers:14dhist,f_blue:phone_bluetooth_doryab_countscansleastfrequentdevicewithinsegmentsothers:14dhist,f_blue:phone_bluetooth_doryab_countscansleastfrequentdeviceacrosssegmentsothers:14dhist,f_blue:phone_bluetooth_doryab_countscansleastfrequentdeviceacrossdatasetothers:14dhist,f_blue:phone_bluetooth_rapids_countscans_norm:14dhist,f_blue:phone_bluetooth_rapids_uniquedevices_norm:14dhist,…,f_blue:phone_bluetooth_doryab_countscansmostfrequentdeviceacrossdatasetothers:weekend,f_blue:phone_bluetooth_doryab_countscansleastfrequentdevicewithinsegmentsothers:weekend,f_blue:phone_bluetooth_doryab_countscansleastfrequentdeviceacrosssegmentsothers:weekend,f_blue:phone_bluetooth_doryab_countscansleastfrequentdeviceacrossdatasetothers:weekend,f_blue:phone_bluetooth_rapids_countscans_norm:weekend,f_blue:phone_bluetooth_rapids_uniquedevices_norm:weekend,f_blue:phone_bluetooth_rapids_countscansmostuniquedevice_norm:weekend,f_blue:phone_bluetooth_doryab_countscansall_norm:weekend,f_blue:phone_bluetooth_doryab_uniquedevicesall_norm:weekend,f_blue:phone_bluetooth_doryab_meanscansall_norm:weekend,f_blue:phone_bluetooth_doryab_stdscansall_norm:weekend,f_blue:phone_bluetooth_doryab_countscansmostfrequentdevicewithinsegmentsall_norm:weekend,f_blue:phone_bluetooth_doryab_countscansmostfrequentdeviceacrosssegmentsall_norm:weekend,f_blue:phone_bluetooth_doryab_countscansmostfrequentdeviceacrossdatasetall_norm:weekend,f_blue:phone_bluetooth_doryab_countscansleastfrequentdevicewithinsegmentsall_norm:weekend,f_blue:phone_bluetooth_doryab_countscansleastfrequentdeviceacrosssegmentsall_norm:weekend,f_blue:phone_bluetooth_doryab_countscansleastfrequentdeviceacrossdatasetall_norm:weekend,f_blue:phone_bluetooth_doryab_countscansown_norm:weekend,f_blue:phone_bluetooth_doryab_uniquedevicesown_norm:weekend,f_blue:phone_bluetooth_doryab_meanscansown_norm:weekend,f_blue:phone_bluetooth_doryab_stdscansown_norm:weekend,f_blue:phone_bluetooth